<!-- image from image -->
![image](images/exemple.png)

## 🎬 **Projet : Recommandation de Films avec Neo4j & Graph Data Science**

### 🔍 Objectif général :

Concevoir un système de recommandation de films en exploitant une base de données orientée graphe. L’algorithme s’appuie sur les notes attribuées par les utilisateurs aux films pour proposer des contenus personnalisés.

---

## 1. 📦 Chargement des données

Les données utilisées proviennent du dataset **MovieLens 1M**, comprenant :

* une liste de films avec titres et genres,
* des utilisateurs avec âge, sexe, profession, etc.,
* les notes attribuées par les utilisateurs aux films.

---

## 2. 🧩 Modélisation dans Neo4j

Les données sont importées dans Neo4j sous forme de graphe avec :

* des **nœuds `User`** pour représenter les utilisateurs,
* des **nœuds `Movie`** pour représenter les films,
* des **relations `RATED`** qui relient un utilisateur à un film avec une note.

Cette modélisation permet d’exprimer facilement les relations complexes comme la co-notation ou les préférences croisées.

---

## 3. 🔄 Recherche de films similaires

L’idée ici est qu’un film est **similaire** à un autre si de nombreux utilisateurs les ont **tous deux bien notés**. On recherche donc des films co-notés positivement par les mêmes utilisateurs, et on les classe par fréquence.

---

## 4. 👥 Similarité entre utilisateurs

On projette un graphe entre les utilisateurs et les films qu’ils ont notés, puis on calcule une **similarité entre utilisateurs** basée sur leurs comportements (notes attribuées).
Cela permet de repérer des **profils proches**.

---

## 5. 🧠 Recommandation collaborative

Une fois les utilisateurs similaires identifiés, on recommande à un utilisateur les films **que ses voisins proches ont bien notés**, mais qu’il n’a pas encore vus.

Les recommandations sont pondérées selon :

* la **note** attribuée par l’utilisateur voisin,
* et la **similarité** entre les deux utilisateurs.

---

## 6. ✅ Vérification des goûts réels

On récupère les films déjà notés par l’utilisateur pour comparer les recommandations à ses préférences connues. Cela permet une **évaluation qualitative** des suggestions proposées.

---

## 7. 🎞️ Recommandation par genre

Une méthode alternative consiste à recommander des films en se basant sur les **genres préférés** de l’utilisateur. On analyse ses notes positives et on en déduit ses genres favoris.
Ensuite, on propose des films du même genre qu’il n’a pas encore vus.

---

## 🧾 Conclusion

Ce projet démontre l'efficacité des bases de données orientées graphes pour la **recommandation personnalisée**, grâce à :

* la flexibilité du modèle graphe,
* les algorithmes de **similarité** (nœuds ou genres),
* et l'exploitation intuitive des relations complexes.

Il peut être étendu à d’autres domaines comme la musique, l’e-commerce ou les réseaux sociaux.

---

Souhaites-tu que je t’ajoute une **introduction générale** ou une **conclusion académique** selon ton niveau d’étude (Master, Licence, etc.) ?


In [10]:
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience

In [11]:
# Neo4j Connection details
DB_ULR = 'bolt://localhost:7687'
DB_USER = 'neo4j' 
DB_PASS = 'test1234'
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))
gds.version()

'2.13.3'

In [12]:
nodes = gds.run_cypher('''
    MATCH (n)
    RETURN COUNT(n)
''') 
nodes.head()

,COUNT(n)
0,9923


In [13]:
movies = pd.read_csv('datasets/movies.dat', sep='::', encoding="ISO-8859-1", names=['MovieID','Title','Genres', 'Actors', 'Realizations', 'Date'], engine='python')
ratings = pd.read_csv('datasets/ratings.dat', sep='::', encoding="ISO-8859-1",names=['UserID','MovieID','Rating','Timestamp'], engine='python')
users = pd.read_csv('datasets/users.dat', sep='::', encoding="ISO-8859-1",names=['UserID','Name','Gender','Age','Occupation','Zip_code'], engine='python')

In [14]:
print("************** Les films *******")
display(movies.head())
print("************** Les utilisateurs *******")
display(users.head())
print("************** Les notes *******")
display(ratings.head())

************** Les films *******


,MovieID,Title,Genres,Actors,Realizations,Date
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,Tina Stewart|Jessica Smith,Jeremy Hendricks|Sonya Edwards,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,Richard Sanchez|David Walker|Carol Rodriguez|J...,Edward Williams|Keith Hudson,1995.0
2,3,Grumpier Old Men,Comedy|Romance,Nicholas Ramsey|Donna Williams|David Huerta|Ch...,Kyle Luna|Daniel Gonzalez,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,Curtis Wright,Derrick Campbell,1995.0
4,5,Father of the Bride Part II,Comedy,David Rogers|Stanley Galloway,Howard Martinez,1995.0


************** Les utilisateurs *******


,UserID,Name,Gender,Age,Occupation,Zip_code
0,1,Mary Campbell,F,35,27,92606
1,2,Larry Shaw,M,15,22,6265
2,3,Suzanne Quinn,F,27,27,54865
3,4,Monique Hughes,F,62,2,1871
4,5,Justin Stone,M,47,21,99193


************** Les notes *******


,UserID,MovieID,Rating,Timestamp
0,99476,104374,3.5,1467897440
1,107979,2634,4.0,994007728
2,155372,1614,3.0,1097887531
3,65225,7153,4.0,1201382275
4,79161,500,5.0,1488915363


In [15]:
def create_user_nodes(gds, users_df):
    '''
    Crée des noeuds d'utilisateur dans la base de données Neo4j.
        :param gds: Instance de GraphDataScience
        :param users_df: DataFrame contenant les informations des utilisateurs
        :return: Résultat de la requête Cypher
    '''
    # Ajouter la contrainte (si elle n'existe pas encore)
    gds.run_cypher('''
        CREATE CONSTRAINT IF NOT EXISTS 
        FOR (n:User) 
        REQUIRE n.id IS NODE KEY
    ''')

    # Inserer les Users
    result = gds.run_cypher('''
        UNWIND $data AS row
        MERGE (u:User {id: row.UserID})
        SET u.gender = row.Gender,
            u.age = row.Age
        RETURN count(*) AS users_created
    ''', params={'data': users_df.to_dict('records')})

    return result


def create_movie_nodes(gds, movies_df):
    '''
    Crée des noeuds de film dans la base de données Neo4j.
        :param gds: Instance de GraphDataScience
        :param movies_df: DataFrame contenant les informations des films
        :return: Résultat de la requête Cypher
    '''
    # Ajouter la contrainte (si elle n'existe pas encore)
    gds.run_cypher('''
        CREATE CONSTRAINT IF NOT EXISTS 
        FOR (n:Movie) 
        REQUIRE n.id IS NODE KEY
    ''')

    # Inserer les Movies
    result = gds.run_cypher('''
        UNWIND $data AS row
        MERGE (m:Movie {id: row.MovieID})
        SET m.title = row.Title,
            m.genres = row.Genres
        RETURN count(*) AS movies_created
    ''', params={'data': movies_df.to_dict('records')})

    return result



In [16]:
# Créer les utilisateurs
res_users = create_user_nodes(gds, users)
print(res_users)

# Créer les films
res_movies = create_movie_nodes(gds, movies)
print(res_movies)


   users_created
0         162541
   movies_created
0           62423


In [17]:
nodes = gds.run_cypher('''
    MATCH (u:User) RETURN count(u);
''') 
nodes.head()

,count(u)
0,162541


In [18]:
nodes = gds.run_cypher('''
    MATCH (m:Movie) RETURN count(m);
''')
nodes.head()

,count(m)
0,62457


In [19]:
def create_rated_relationships(gds, ratings_df, chunk_size=200):
    i = 1
    for chunk in np.array_split(ratings_df, chunk_size):
        print(f"Chunk {i}/{chunk_size}")
        result = gds.run_cypher('''
            UNWIND $data AS row
            MATCH (u:User {id: row.UserID}), (m:Movie {id: row.MovieID})
            MERGE (u)-[r:RATED]->(m)
            SET r.rating = row.Rating
            RETURN count(*) AS created
        ''', params={'data': chunk.to_dict('records')})
        print(result.head())
        i += 1
    return result

In [20]:
# Créer les relations RATED
res_rated = create_rated_relationships(gds, ratings)
print(res_rated)

Chunk 1/200
   created
0       50
Chunk 2/200


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


   created
0       50
Chunk 3/200
   created
0       50
Chunk 4/200
   created
0       50
Chunk 5/200
   created
0       50
Chunk 6/200
   created
0       50
Chunk 7/200
   created
0       50
Chunk 8/200
   created
0       50
Chunk 9/200
   created
0       50
Chunk 10/200
   created
0       50
Chunk 11/200
   created
0       50
Chunk 12/200
   created
0       50
Chunk 13/200
   created
0       50
Chunk 14/200
   created
0       50
Chunk 15/200
   created
0       50
Chunk 16/200
   created
0       50
Chunk 17/200
   created
0       50
Chunk 18/200
   created
0       50
Chunk 19/200
   created
0       50
Chunk 20/200
   created
0       50
Chunk 21/200
   created
0       50
Chunk 22/200
   created
0       50
Chunk 23/200
   created
0       50
Chunk 24/200
   created
0       50
Chunk 25/200
   created
0       50
Chunk 26/200
   created
0       50
Chunk 27/200
   created
0       50
Chunk 28/200
   created
0       50
Chunk 29/200
   created
0       50
Chunk 30/200
   created
0       50
Chunk

In [21]:
nodes = gds.run_cypher('''
    MATCH (u:User)-[r:RATED]->(m:Movie) RETURN count(r);
''')
nodes.head()

,count(r)
0,1010173


<!-- image from image -->
![image](images/view_relation.png)

In [22]:
def get_similar_movies(title):
    """
    Récupère la liste des films similaires à un film donné en fonction des utilisateurs 
    ayant noté les deux films avec la note maximale (5).

    Cette fonction interroge la base de données Neo4j pour trouver les films qui partagent 
    des utilisateurs en commun ayant attribué une note de 5 à ces films et au film cible.

    Args:
        title (str): Le titre exact du film pour lequel trouver des films similaires.

    Returns:
        pd.DataFrame: Un DataFrame contenant :
            - title (str): Le titre des films similaires.
            - genres (str): Les genres des films similaires.
            - common_users (int): Le nombre d'utilisateurs ayant noté les deux films avec la note 5.
    """
    
    query = '''
    MATCH (m1:Movie)-[r1:RATED]-(u:User)-[r2:RATED]-(m2:Movie)
    WHERE m1.title = $title
      AND m2.title <> $title
      AND r1.rating = 5 
      AND r2.rating = 5
    RETURN m2.title AS title, m2.genres AS genres, count(DISTINCT u) AS common_users
    ORDER BY common_users DESC
    '''

    result = gds.run_cypher(query, params={'title': title})
    print(f"******** Les films similaires à {title} ******** ")
    return result


similar_movies = get_similar_movies('1408')
similar_movies.head(10)


******** Les films similaires à 1408 ******** 


,title,genres,common_users


In [23]:
def get_similar_movies(title, min_rating=5):
    """
    Retourne les films similaires basés sur les utilisateurs ayant noté les deux films avec une note minimale.

    Args:
        title (str): Le titre du film pour lequel on veut trouver des films similaires.
        min_rating (int): La note minimale que les utilisateurs doivent avoir donnée pour être considérés.

    Returns:
        pd.DataFrame: Liste des films similaires avec le nombre d'utilisateurs en commun.
    """
    
    query = '''
    MATCH (m1:Movie)-[r1:RATED]-(u:User)-[r2:RATED]-(m2:Movie)
    WHERE m1.title = $title
      AND m2.title <> $title
      AND r1.rating >= $min_rating 
      AND r2.rating >= $min_rating
    RETURN m2.title AS title, m2.genres AS genres, count(DISTINCT u) AS common_users
    ORDER BY common_users DESC
    '''

    result = gds.run_cypher(query, params={'title': title, 'min_rating': min_rating})
    print(f"******** Les films similaires à {title} avec une note minimale de {min_rating} ******** ")
    return result


similar_movies = get_similar_movies('I Am Legend', min_rating=4)
similar_movies.head(10)


******** Les films similaires à I Am Legend avec une note minimale de 4 ******** 


,title,genres,common_users


In [24]:
gds.run_cypher("CALL gds.graph.drop('myGraph', false)")

create_projection = gds.run_cypher('''
CALL gds.graph.project(
  'myGraph',
  ['User', 'Movie'],
  {
    RATED: {
      properties: 'rating'
    }
  }
);
''')

create_projection.head()


,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'User': {'label': 'User', 'properties': {}}, ...","{'RATED': {'aggregation': 'DEFAULT', 'orientat...",myGraph,224998,1010173,702


In [25]:
gds.run_cypher("CALL gds.graph.drop('myGraphFiltered', false)")

create_projection = gds.run_cypher('''
CALL gds.graph.project.cypher(
  'myGraphFiltered',
  '
  MATCH (u:User)
  WHERE COUNT { (u)-[:RATED]->() } >= 1000
  RETURN id(u) AS id
  ',
  '
  MATCH (u1:User)-[r:RATED]->(m:Movie)
  RETURN id(u1) AS source, id(m) AS target, r.rating AS rating
  ',
  {validateRelationships: false}
)
''')

create_projection.head()


,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
0,MATCH (u:User)\n WHERE COUNT { (u)-[:RATED]->...,MATCH (u1:User)-[r:RATED]->(m:Movie)\n RETURN...,myGraphFiltered,41,0,331


In [26]:
# MATCH (u:User {id: 1})-[:RATED]->(m1:Movie)
# WITH collect(DISTINCT m1.genres) AS genres_list
# UNWIND genres_list AS genre
# MATCH (m2:Movie)
# WHERE genre IN m2.genres AND NOT EXISTS {
#     MATCH (:User {id: 1})-[:RATED]->(m2)
# }
# RETURN m2.title, m2.genres
# LIMIT 5

# get user similarity
def get_user_similarity(user_id: int) -> pd.DataFrame:
    """
    Récupère les utilisateurs similaires à un utilisateur donné en utilisant l'algorithme 
    de similarité de noeuds GDS (Node Similarity) sur la projection 'myGraphFiltered'.

    Args:
        user_id (int): L'ID de l'utilisateur (attribut logique 'id').

    Returns:
        pd.DataFrame: Un DataFrame contenant :
            - similar_user_id (int) : ID logique de l'utilisateur similaire.
            - similarity (float) : Score de similarité.
    """
    # Étape 1 : Récupérer l'ID interne Neo4j à partir de l'ID logique
    node_id_query = '''
    MATCH (u:User {id: $user_id})
    RETURN id(u) AS nodeId
    '''
    node_id_df = gds.run_cypher(node_id_query, params={'user_id': user_id})
    
    if node_id_df.empty:
        print(f"⚠️ Aucun utilisateur trouvé avec l'id logique {user_id}")
        return pd.DataFrame()

    node_id = node_id_df.iloc[0]['nodeId']

    # Étape 2 : Calculer les similarités avec l'algorithme Node Similarity
    similarity_query = '''
    CALL gds.nodeSimilarity.stream('myGraphFiltered')
    YIELD node1, node2, similarity
    WHERE node1 = $node_id OR node2 = $node_id
    RETURN 
        CASE 
            WHEN node1 = $node_id THEN gds.util.asNode(node2).id 
            ELSE gds.util.asNode(node1).id 
        END AS similar_user_id,
        similarity
    ORDER BY similarity DESC
    LIMIT 10
    '''
    result_df = gds.run_cypher(similarity_query, params={'node_id': node_id})

    print(f"✅ Utilisateurs similaires à l'utilisateur logique {user_id} :")
    return result_df

similar_users = get_user_similarity(3)
similar_users.head(10)

✅ Utilisateurs similaires à l'utilisateur logique 3 :


,similar_user_id,similarity


In [27]:
# Calcul de la similarité entre tous les utilisateurs du graphe 'myGraphFiltered'
query = '''
CALL gds.nodeSimilarity.stream('myGraphFiltered')
YIELD node1, node2, similarity
RETURN 
    gds.util.asNode(node1).id AS user_id_1,
    gds.util.asNode(node2).id AS user_id_2,
    similarity
ORDER BY similarity DESC, user_id_1, user_id_2
'''

# Exécution de la requête Cypher
users_similarity = gds.run_cypher(query)

# Affichage des premières lignes du DataFrame
users_similarity.head()


,user_id_1,user_id_2,similarity


In [28]:
# Create Similar relationship
i=1
for chunk in np.array_split(users_similarity.query('UserID1>UserID2'),10):
  print(i)
  create_similar = gds.run_cypher('''
    unwind $data as row
    match (u1:User{id: row.UserID1}), (u2:User{id: row.UserID2})
    merge (u1)-[r:SIMILAR]->(u2)
    set r.Similarity=row.similarity
    return count(*) as create_rated
    ''', params = {'data': chunk.to_dict('records')})
  i = i+1
create_similar.head()

UndefinedVariableError: name 'UserID1' is not defined

In [ ]:
# Check similar movies
similar_movies_for_user = gds.run_cypher('''
    MATCH (u1:User)-[r1:SIMILAR]-(u2)-[r2:RATED]-(m:Movie)
    WHERE id(u1)=$id
    AND NOT ( (u1)-[]-(m))
    RETURN m.Title,m.Genres,Sum(r1.Similarity*r2.Rating)/sum(r1.Similarity)+log(count(r2)) as score
    ORDER BY score DESC
''',params = {'id':4725})
similar_movies_for_user.head(10)

In [ ]:
# Check actual movies
movies_for_user = gds.run_cypher('''
    MATCH (u1:User)-[r:RATED]-(m:Movie)
  WHERE id(u1)=$id
  RETURN m.Title,m.Genres,r.Rating as rating
  ORDER BY rating DESC
''',params = {'id':4725})
movies_for_user.head(10)